<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from tensorflow import keras

# DenseNet

## The stem component

In [22]:
def stem(inputs, n_filters):
  """ Build the stem component of the DenseNet
      inputs   : represents the input tensor
      n_filters: number of filters for the dense blocks (k)
  """
  # pad the input from 224x224 to 230x230
  x = keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)

  # first large convolution for abstract features for input 224 x 224 and output 112 x 112
  # the stem convolution uses 2 * k (growth rate) number of filters
  x = keras.layers.Conv2D(filters=2 * n_filters, kernel_size=7, strides=2, use_bias=False, kernel_initializer='he_normal')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # add padding so when downsampling we fit shape 56 x 56
  x = keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
  x = keras.layers.MaxPooling2D((3, 3), strides=2)(x)

  return x

## The learner component

### The Transition Block

In [23]:
def trans_block(x, reduction):
  """ Build a transition block
      x        : input layer
      reduction: percentage of reduction of feature maps
  """

  # calculate the number of filters
  # we reduce (compress) the number of feature maps (DenseNet-C)
  # 'shape[n]' returns a class object. We use int() to cast it into the dimension size
  n_filters = int( int(x.shape[3]) * reduction)

  # a 1x1 linear projection convolution
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Conv2D(filters=n_filters, kernel_size=1, strides=1, use_bias=False, kernel_initializer='he_normal')(x)

  # we use mean value (average) instead of max value sampling when pooling reduce by 75%
  x = keras.layers.AveragePooling2D(pool_size=2, strides=2)(x)

  return x

### The Dense Block

In [24]:
def dense_block(x, n_filters):
  """ Build a densely connected residual block
      x        : input to the block
      n_filters: number of filters in convolution layer in residual block
  """
  # save the input tensor into residual block
  shortcut = x

  # a 1x1 conv layer for dimensionality expansion, expand filters by 4 (DenseNet-B)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  x = keras.layers.Conv2D(filters=4 * n_filters, kernel_size=1, strides=1, use_bias=False, kernel_initializer='he_normal')(x)

  # our bottleneck convolution
  # a 3x3 convolution with padding=same to preserve same shape of feature maps
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)
  x = keras.layers.Conv2D(filters=n_filters, kernel_size=3, strides=1, padding='same', use_bias=False, kernel_initializer='he_normal')(x)

  # concatenate the input (identity) with the output of the residual block
  # concatenation (vs. merging) provides Feature Reuse between layers
  x = keras.layers.Concatenate()([shortcut, x])


  return x

### The DenseNet Group

In [25]:
def group(x, n_blocks, n_filters, reduction=None):
  """ Build a dense group.
      x         : input to the group
      n_blocks  : number of residual blocks in dense block
      n_filters : number of filters in convolution layer in residual block
      reduction : amount to reduce feature maps by
  """
  # we construct a group of densely connected residual blocks
  for _ in range(n_blocks):
    x = dense_block(x, n_filters)

  # we construct interceding transition block
  if reduction is not None:
    x = trans_block(x, reduction)

  return x

### Putting it all together

In [26]:
def learner(x, groups, n_filters, reduction):
  """ Build the learner component of the DenseNet
      x         : input to the learner
      groups    : set of number of blocks per group
      n_filters : number of filters (growth rate)
      reduction : the amount to reduce (compress) feature maps by
  """
  # pop off the list the last dense block
  last = groups.pop()

  # we create the dense groups and interceding transition blocks
  for n_blocks in groups:
    x = group(x, n_blocks, n_filters, reduction)

  # we add the last dense group w/o a following transition block
  x = group(x, last, n_filters)

  return x

## The task component

In [27]:
def task(x, n_classes):
  """ Build the task component of DenseNet.
      x         : input to the classifier
      n_classes : number of output classes
  """

  # Global Average Pooling will flatten the 7x7 feature maps into 1D feature maps
  x = keras.layers.GlobalAveragePooling2D()(x)
  # Fully connected output layer (classification)
  x = keras.layers.Dense(units=n_classes, activation='softmax', kernel_initializer='he_normal')(x)

  return x

## DenseNet model

In [28]:
# Meta-parameter: amount to reduce feature maps by (compression factor) during transition blocks
reduction = 0.5

# Meta-parameter: number of filters in a convolution block within a residual block (growth rate)
n_filters = 32

# Meta-parameter: number of residual blocks in each dense group
groups = { 121 : [6, 12, 24, 16],	# DenseNet 121
           169 : [6, 12, 32, 32],	# DenseNet 169
           201 : [6, 12, 48, 32] }	# DenseNet 201


In [29]:
def build_densenet(groups, shape=(224, 224, 3), reduction = 0.5, n_filters = 32, classes=1000):
  # the input vector
  inputs = keras.Input(shape=shape)
  # stem component
  x = stem(inputs, n_filters)
  # the learner component
  x = learner(x, groups[121], n_filters, reduction)
  # the task component
  outputs = task(x, classes)

  # return the modle
  return keras.Model(inputs=inputs, outputs=outputs, name="DenseNet")

In [30]:
# create the DenseNet model
densenet_model = build_densenet(groups)

# print a summary of our DenseNet model
densenet_model.summary()

Model: "DenseNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_3[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv2d_120 (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 batch_normalization_120 (B  (None, 112, 112, 64)         256       ['conv2d_120[0][0]']   